In [8]:
import pandas as pd 
import re
pathmoe = '/hdd3/sonia/be_great/ckpts/moe/dgpt2/adult-4col/jul16/samples'
pathoh = '/hdd3/sonia/be_great/ckpts/dgpt2/adult-4col/samples'
pathreal = '/hdd3/sonia/data/adult.csv'
path = pathmoe

In [9]:
patternmoe = f'\<\|endoftext\|\>age is (\d+)\.<EOS>workclass is (\w+)\.<EOS>fnlwgt is (\d+).+<EOS>income is (.+)\.<EOS>'
patternoh = f'\<\|endoftext\|\>age is (\d+)\.<EOS> workclass is (\w+)\.<EOS> fnlwgt is (\d+).+<EOS> income is (.+)\.<EOS>'
pattern = patternmoe

In [10]:
with open(path+'.txt', 'r') as f:
    raws = f.readlines()

parsed = []
for raw in raws:
    vals = re.findall(pattern, raw)
    if len(vals) == 1:
        parsed.append(vals[0])
len(parsed)

6633

In [11]:
df = pd.DataFrame(parsed, columns=['age', 'workclass', 'fnlwgt', 'income'])
df.head()

,age,workclass,fnlwgt,income
0,50,Private,4781887,<=50K
1,44,Private,17126743,<=50K
2,30,Private,40,<=50K
3,32,Private,2538,<=50K
4,38,Private,45624,<=50K


In [12]:
real = pd.read_csv(pathreal)
real = real[df.columns]
ords = ['workclass', 'income']
ordvals = {col:set(real[col].unique()) for col in ords}
for col in ordvals:
    ordvals[col] = [val.strip() for val in ordvals[col]]
ordvals

{'workclass': ['Federal-gov',
  'Without-pay',
  'Self-emp-not-inc',
  'State-gov',
  '?',
  'Private',
  'Self-emp-inc',
  'Never-worked',
  'Local-gov'],
 'income': ['>50K', '<=50K']}

In [13]:
df=df[df['workclass'].isin(ordvals['workclass']) & df['income'].isin(ordvals['income'])]
df.head()

,age,workclass,fnlwgt,income
0,50,Private,4781887,<=50K
1,44,Private,17126743,<=50K
2,30,Private,40,<=50K
3,32,Private,2538,<=50K
4,38,Private,45624,<=50K


In [14]:
df.to_csv(path+'.csv', index=False)